In [1]:
import os
os.chdir('..')

In [2]:
from models.bailian import LLM

# export API_KEY='xxx'
llm = LLM(api_key=os.environ.get('API_KEY'))

In [3]:
os.environ['LANG'] = 'ch'
from templates.make_prompts import *
import json

output_json = {
    "title": "example title",
    "pages": [
        {
            "title": "title for page 1",
            "content": [
                {
                    "title": "title for paragraph 1",
                    "description": "detail for paragraph 1",
                },
                {
                    "title": "title for paragraph 2",
                    "description": "detail for paragraph 2",
                },
            ],
        },
        {
            "title": "title for page 2",
            "content": [
                {
                    "title": "title for paragraph 1",
                    "description": "detail for paragraph 1",
                },
                {
                    "title": "title for paragraph 2",
                    "description": "detail for paragraph 2",
                },
                {
                    "title": "title for paragraph 3",
                    "description": "detail for paragraph 3",
                },
            ],
        },
    ],
}

def make_prompt(topic, pages):
    output_format = json.dumps(output_json, ensure_ascii=False)
    prompt = make_skeleton(
        additional_prompts=[
            make_role('文档编写员'),
            f'我要准备1个主题关于"{topic}"的PPT，要求一共写{pages}页，请你根据主题生成详细内容，不要省略。',
            f'\n按这个JSON格式输出\n{output_format}\n只能返回JSON\n',
        ]
    )

    return prompt

In [4]:
import re


def parse_content(content):
    if '```json' in content:
        ret = re.search('```json\n(.+)\n```', content, flags=re.S).group(1)
    else:
        ret = content

    ret = json.loads(ret)
    return ret

In [5]:
# pip install python-pptx
from pptx import Presentation

def parse_ret(ret, save_fp):
    ppt = Presentation()

    # PPT首页
    slide = ppt.slides.add_slide(ppt.slide_layouts[0])  # title&subtitle layout
    slide.placeholders[0].text = ret['title']

    # 内容页
    for i, page in enumerate(ret['pages']):
        slide = ppt.slides.add_slide(ppt.slide_layouts[1])  # title&content layout
        # 标题
        slide.placeholders[0].text = page['title']
        # 正文
        for sub_content in page['content']:
            # 一级正文
            sub_title = slide.placeholders[1].text_frame.add_paragraph()
            sub_title.text, sub_title.level = sub_content['title'], 1
            # 二级正文
            sub_description = slide.placeholders[1].text_frame.add_paragraph()
            sub_description.text, sub_description.level = sub_content['description'], 2

    ppt.save(save_fp)
    print(f'Save to {save_fp} successfully')

In [6]:
def run(topic, pages):
    prompt = make_prompt(topic, pages)
    content = llm(prompt)
    ret = parse_content(content)
    parse_ret(ret, f'temp/{topic}.pptx')
    return prompt, ret

In [7]:
prompt, ret = run('如何学习大模型', 4)
print(prompt)
print('-------')
print(json.dumps(ret, indent=4, ensure_ascii=False))

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Save to temp/如何学习大模型.pptx successfully

现在你扮演一名文档编写员。我要准备1个主题关于"如何学习大模型"的PPT，要求一共写4页，请你根据主题生成详细内容，不要省略。
按这个JSON格式输出
{"title": "example title", "pages": [{"title": "title for page 1", "content": [{"title": "title for paragraph 1", "description": "detail for paragraph 1"}, {"title": "title for paragraph 2", "description": "detail for paragraph 2"}]}, {"title": "title for page 2", "content": [{"title": "title for paragraph 1", "description": "detail for paragraph 1"}, {"title": "title for paragraph 2", "description": "detail for paragraph 2"}, {"title": "title for paragraph 3", "description": "detail for paragraph 3"}]}]}
只能返回JSON

开始!


-------
{
    "title": "如何学习大模型",
    "pages": [
        {
            "title": "第1页：大模型概述",
            "content": [
                {
                    "title": "大模型定义",
                    "description": "大模型，又称大规模预训练模型，是指通过海量数据训练得到的具有强大语言理解与生成能力的AI模型，如GPT系列、BERT等。这些模型能够执行多样化的任务，从文本生成到问答系统，极大地推动了人工智能领域的发展。"
                },
                {
      